In [1]:
# Cell 1: Import libraries and set dataset path (Member 1)
import pandas as pd
import numpy as np

pd.set_option('display.float_format', lambda x: f'{x:,.3f}')

# Notebook is inside project/code
traffic_path = "../dataset/Traffic_Flow_Data_Jan_to_June_2023_SDCC (2).csv"

print("Using traffic file:", traffic_path)


Using traffic file: ../dataset/Traffic_Flow_Data_Jan_to_June_2023_SDCC (2).csv


In [2]:
# Cell 2: Load raw traffic flow dataset
df = pd.read_csv(traffic_path)

print("Raw dataset shape:", df.shape)
display(df.head())
display(df.info())


Raw dataset shape: (1048575, 12)


,site,day,date,start_time,end_time,flow,flow_pc,cong,cong_pc,dsat,dsat_pc,ObjectId
0,N01111A,TH,05/01/2023,2023/07/04 08:30:00+00,09:45,0,0,0,0,50,98,1
1,N01111A,WE,04/01/2023,2023/07/04 07:30:00+00,08:45,300,100,0,100,22,98,2
2,N01111A,FR,06/01/2023,2023/07/04 09:30:00+00,10:45,445,100,0,100,31,100,3
3,N01111A,TU,03/01/2023,2023/07/04 06:30:00+00,07:45,0,0,0,0,11,32,4
4,N01111A,TH,05/01/2023,2023/07/04 08:45:00+00,10:00,0,0,0,0,50,107,5


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 12 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   site        1048575 non-null  object
 1   day         1048575 non-null  object
 2   date        1048575 non-null  object
 3   start_time  1048575 non-null  object
 4   end_time    1048575 non-null  object
 5   flow        1048575 non-null  int64 
 6   flow_pc     1048575 non-null  int64 
 7   cong        1048575 non-null  int64 
 8   cong_pc     1048575 non-null  int64 
 9   dsat        1048575 non-null  int64 
 10  dsat_pc     1048575 non-null  int64 
 11  ObjectId    1048575 non-null  int64 
dtypes: int64(7), object(5)
memory usage: 96.0+ MB


None

In [3]:
# Cell 3: Clean column names for consistency
df.columns = [c.strip().lower() for c in df.columns]

print("Column names after cleaning:")
print(df.columns.tolist())
display(df.head())


Column names after cleaning:
['site', 'day', 'date', 'start_time', 'end_time', 'flow', 'flow_pc', 'cong', 'cong_pc', 'dsat', 'dsat_pc', 'objectid']


,site,day,date,start_time,end_time,flow,flow_pc,cong,cong_pc,dsat,dsat_pc,objectid
0,N01111A,TH,05/01/2023,2023/07/04 08:30:00+00,09:45,0,0,0,0,50,98,1
1,N01111A,WE,04/01/2023,2023/07/04 07:30:00+00,08:45,300,100,0,100,22,98,2
2,N01111A,FR,06/01/2023,2023/07/04 09:30:00+00,10:45,445,100,0,100,31,100,3
3,N01111A,TU,03/01/2023,2023/07/04 06:30:00+00,07:45,0,0,0,0,11,32,4
4,N01111A,TH,05/01/2023,2023/07/04 08:45:00+00,10:00,0,0,0,0,50,107,5


In [4]:
# Cell 4: Parse actual date and start time columns
df["date"] = pd.to_datetime(df["date"], dayfirst=True, errors="coerce").dt.date
df["start_time"] = pd.to_datetime(df["start_time"], errors="coerce")

print("Invalid date count:", df["date"].isna().sum())
display(df[["site", "day", "date", "start_time"]].head(10))


Invalid date count: 0


,site,day,date,start_time
0,N01111A,TH,2023-01-05,2023-07-04 08:30:00+00:00
1,N01111A,WE,2023-01-04,2023-07-04 07:30:00+00:00
2,N01111A,FR,2023-01-06,2023-07-04 09:30:00+00:00
3,N01111A,TU,2023-01-03,2023-07-04 06:30:00+00:00
4,N01111A,TH,2023-01-05,2023-07-04 08:45:00+00:00
5,N01111A,TH,2023-01-05,2023-07-04 09:00:00+00:00
6,N01111A,WE,2023-01-04,2023-07-04 07:45:00+00:00
7,N01111A,TU,2023-01-03,2023-07-04 06:45:00+00:00
8,N01111A,TH,2023-01-05,2023-07-04 09:15:00+00:00
9,N01111A,WE,2023-01-04,2023-07-04 08:00:00+00:00


In [5]:
# Cell 5: Remove invalid dates and convert flow to numeric
before = df.shape[0]
df = df.dropna(subset=["date"])
after = df.shape[0]

print(f"Rows dropped due to invalid dates: {before - after}")

df["flow"] = pd.to_numeric(df["flow"], errors="coerce")

display(df["flow"].describe())
print("Missing flow values:", df["flow"].isna().sum())


Rows dropped due to invalid dates: 0


count   1,048,575.000
mean          133.005
std           210.080
min             0.000
25%             0.000
50%            16.000
75%           191.000
max         2,142.000
Name: flow, dtype: float64

Missing flow values: 0


In [6]:
# Cell 6: Remove missing, negative and extreme flow values
before = df.shape[0]
df = df.dropna(subset=["flow"])
after = df.shape[0]
print(f"Rows dropped due to missing flow: {before - after}")

before = df.shape[0]
df = df[df["flow"] >= 0]
after = df.shape[0]
print(f"Rows dropped due to negative flow: {before - after}")

flow_threshold = df["flow"].quantile(0.999)
before = df.shape[0]
df = df[df["flow"] <= flow_threshold]
after = df.shape[0]

print("Outlier threshold:", flow_threshold)
print(f"Rows dropped as outliers: {before - after}")


Rows dropped due to missing flow: 0
Rows dropped due to negative flow: 0
Outlier threshold: 1121.0
Rows dropped as outliers: 1045


In [7]:
# Cell 7: Create time-based features for later analysis
df["hour"] = df["start_time"].dt.hour
df["month"] = df["start_time"].dt.month
df["day_of_week_num"] = df["start_time"].dt.weekday
df["is_weekend"] = df["day_of_week_num"].isin([5, 6]).astype(int)

df["is_peak"] = (
    df["hour"].between(7, 10) | df["hour"].between(16, 19)
).astype(int)

display(df[["start_time", "date", "hour", "day_of_week_num", "is_weekend", "is_peak"]].head(10))


,start_time,date,hour,day_of_week_num,is_weekend,is_peak
0,2023-07-04 08:30:00+00:00,2023-01-05,8,1,0,1
1,2023-07-04 07:30:00+00:00,2023-01-04,7,1,0,1
2,2023-07-04 09:30:00+00:00,2023-01-06,9,1,0,1
3,2023-07-04 06:30:00+00:00,2023-01-03,6,1,0,0
4,2023-07-04 08:45:00+00:00,2023-01-05,8,1,0,1
5,2023-07-04 09:00:00+00:00,2023-01-05,9,1,0,1
6,2023-07-04 07:45:00+00:00,2023-01-04,7,1,0,1
7,2023-07-04 06:45:00+00:00,2023-01-03,6,1,0,0
8,2023-07-04 09:15:00+00:00,2023-01-05,9,1,0,1
9,2023-07-04 08:00:00+00:00,2023-01-04,8,1,0,1


In [8]:
# Cell 8: Aggregate to daily level (this creates traf_daily)
traf_daily = (
    df.groupby("date")
      .agg(
          daily_flow_total=("flow", "sum"),
          daily_flow_mean=("flow", "mean"),
          daily_cong_mean=("cong", "mean"),
          daily_dsat_mean=("dsat", "mean"),
          num_records=("flow", "count")
      )
      .reset_index()
      .sort_values("date")
)

print("Daily dataset shape:", traf_daily.shape)
display(traf_daily.head(15))


Daily dataset shape: (177, 6)


,date,daily_flow_total,daily_flow_mean,daily_cong_mean,daily_dsat_mean,num_records
0,2023-01-03,649194,153.765,0.208,27.207,4222
1,2023-01-04,551602,90.190,0.173,21.307,6116
2,2023-01-05,464146,75.903,0.309,24.752,6115
3,2023-01-06,860701,140.937,0.490,25.563,6107
4,2023-01-07,713575,116.388,0.295,21.667,6131
5,2023-01-08,582720,95.045,0.157,17.114,6131
6,2023-01-09,817644,133.799,0.447,23.800,6111
7,2023-01-10,848945,138.966,0.810,24.911,6109
8,2023-01-11,885515,144.787,0.642,26.035,6116
9,2023-01-12,887731,145.078,0.660,26.149,6119


In [9]:
# Cell 9: Save cleaned daily traffic dataset
out_path = "../cleaned dataset/traffic_daily_cleaned_member1_v2.csv"
traf_daily.to_csv(out_path, index=False)

print("Saved cleaned dataset to:", out_path)


Saved cleaned dataset to: ../cleaned dataset/traffic_daily_cleaned_member1_v2.csv


In [10]:
# Cell: Import model libraries (required for training)
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


In [11]:
# Cell: Load merged dataset 
data_path =  "../cleaned dataset/merged_traffic_weather_main_dataset_clean.csv"

print("Merged dataset path:", data_path)

df = pd.read_csv(data_path)

# Making sure date is valid and sorted 
df["date"] = pd.to_datetime(df["date"], errors="coerce")
df = df.dropna(subset=["date"]).sort_values("date").reset_index(drop=True)

print("Merged dataset shape:", df.shape)
display(df.head())

Merged dataset path: ../cleaned dataset/merged_traffic_weather_main_dataset_clean.csv
Merged dataset shape: (177, 13)


,date,daily_flow_total,daily_flow_mean,daily_cong_mean,daily_dsat_mean,num_records,rain_mm,tmax_c,tmin_c,tmean,wind_speed_knots,sunshine_hours,soil_temp_c
0,2023-01-03,649194,153.765,0.208,27.207,4222,1.500,13.300,5.600,9.450,12.500,0.000,6.125
1,2023-01-04,551602,90.190,0.173,21.307,6116,0.100,12.600,8.800,10.700,13.700,0.500,8.200
2,2023-01-05,464146,75.903,0.309,24.752,6115,2.700,12.600,6.700,9.650,10.300,0.000,8.025
3,2023-01-06,860701,140.937,0.490,25.563,6107,2.300,11.500,5.600,8.550,9.700,0.200,6.325
4,2023-01-07,713575,116.388,0.295,21.667,6131,1.900,10.700,6.400,8.550,12.100,5.200,6.675


In [12]:
print(df.columns.tolist())


['date', 'daily_flow_total', 'daily_flow_mean', 'daily_cong_mean', 'daily_dsat_mean', 'num_records', 'rain_mm', 'tmax_c', 'tmin_c', 'tmean', 'wind_speed_knots', 'sunshine_hours', 'soil_temp_c']


In [13]:
# Cell: Features and target
target_col = "daily_flow_total"
X = df.drop(columns=["date", target_col])
y = df[target_col]

print("X shape:", X.shape)
print("y shape:", y.shape)

X shape: (177, 11)
y shape: (177,)


In [14]:
# Cell: Time-aware train/test split (80/20)
split_idx = int(len(df) * 0.8)

X_train = X.iloc[:split_idx]
y_train = y.iloc[:split_idx]

X_test = X.iloc[split_idx:]
y_test = y.iloc[split_idx:]

print("Train size:", X_train.shape[0])
print("Test size :", X_test.shape[0])
print("Train dates:", df["date"].iloc[0], "to", df["date"].iloc[split_idx - 1])
print("Test dates :", df["date"].iloc[split_idx], "to", df["date"].iloc[-1])


Train size: 141
Test size : 36
Train dates: 2023-01-03 00:00:00 to 2023-05-25 00:00:00
Test dates : 2023-05-26 00:00:00 to 2023-06-30 00:00:00


In [15]:
# Model 1 – Linear Regression
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred_lr = lr.predict(X_test)

from sklearn.metrics import root_mean_squared_error, mean_absolute_error, r2_score

rmse_lr = root_mean_squared_error(y_test, y_pred_lr)
mae_lr  = mean_absolute_error(y_test, y_pred_lr)
r2_lr   = r2_score(y_test, y_pred_lr)

print("\nLinear Regression Results")
print(f"RMSE: {rmse_lr:,.3f}")
print(f"MAE : {mae_lr:,.3f}")
print(f"R²  : {r2_lr:.3f}")


Linear Regression Results
RMSE: 30,049.773
MAE : 25,385.371
R²  : 0.926


In [16]:

# Cell: Model 2 — Random Forest
rf = RandomForestRegressor(
    n_estimators=300,
    max_depth=None,
    random_state=42,
    n_jobs=-1
)

rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

rmse_rf = mean_squared_error(y_test, y_pred_rf, squared=False)
mae_rf = mean_absolute_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)


c:\Users\siddh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [17]:
print("\nRandom Forest Results")
print(f"RMSE: {rmse_rf:,.3f}")
print(f"MAE : {mae_rf:,.3f}")
print(f"R²  : {r2_rf:,.3f}")

# Cell: Quick comparison
print("\nModel Comparison")
print(f"Linear Regression -> RMSE: {rmse_lr:,.3f}, R²: {r2_lr:,.3f}")
print(f"Random Forest     -> RMSE: {rmse_rf:,.3f}, R²: {r2_rf:,.3f}")


Random Forest Results
RMSE: 10,584.280
MAE : 7,448.792
R²  : 0.991

Model Comparison
Linear Regression -> RMSE: 30,049.773, R²: 0.926
Random Forest     -> RMSE: 10,584.280, R²: 0.991
